In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import ast
import networkx as nx
from pyvis.network import Network
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
import numba
import seaborn as sns
import iqplot
from sklearn import metrics
import bokeh.models
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
gnnwisuncertainty = pd.read_csv('data/Heterophilic/WisconsinGNNRunConsistancyProbsVSGroundTruth.csv')
gnnwisgraph = gnnwisuncertainty[gnnwisuncertainty['Run'] == 39]
betawisuncertainty = pd.read_csv('data/Heterophilic/WisconsinBetaRunConsistancyProbsVSGroundTruth.csv')
betawisgraph = betawisuncertainty[betawisuncertainty['Run'] == 22]
gnntexuncertainty = pd.read_csv('data/Heterophilic/TexasGNNRunConsistancyProbsVSGroundTruth.csv')
gnntexgraph = gnntexuncertainty[gnntexuncertainty['Run'] == 11]
betatexuncertainty = pd.read_csv('data/Heterophilic/TexasBetaRunConsistancyProbsVSGroundTruth.csv')
betatexgraph = betatexuncertainty[betatexuncertainty['Run'] == 1]
wisout = np.load('data/Other/WisconsinNodeLabels.npy')
texout = np.load('data/Other/TexasNodeLabels.npy')

In [3]:
betawisres = pd.read_csv('data/Heterophilic/TexasBetaRunConsistancy.csv')
betawisres = betawisres[betawisres['Run'] == 1]
wp = 0.5
wn = 0.5
betawisres['Num'] = (wp + wn) * betawisres['Phen Pos Fidelity'] * (1 - betawisres['Phen Neg Fidelity'])
betawisres['Denom'] = wp * (1 - betawisres['Phen Neg Fidelity']) + wn * betawisres['Phen Pos Fidelity']
betawisres['Phenomenon Characterization Score'] = betawisres['Num'] / betawisres['Denom']
betawisres['Num'] = (wp + wn) * betawisres['Model Pos Fidelity'] * (1 - betawisres['Model Neg Fidelity'])
betawisres['Denom'] = wp * (1 - betawisres['Model Neg Fidelity']) + wn * betawisres['Model Pos Fidelity']
betawisres['Model Characterization Score'] = betawisres['Num'] / betawisres['Denom']
betawisres.head()

,Dataset,Explainer,Run,Phen Pos Fidelity,Phen Neg Fidelity,Model Pos Fidelity,Model Neg Fidelity,Unfaithfulness,Exp Size Percent,Num,Denom,Phenomenon Characterization Score,Model Characterization Score
0,Texas,Beta,1,0.032787,0.005464,0.038251,0.005464,1.0,0.455385,0.038042,0.516393,0.063481,0.073669


In [4]:
betawisres = pd.read_csv('data/Heterophilic/TexasGNNRunConsistancy.csv')
betawisres = betawisres[betawisres['Run'] == 11]
wp = 0.5
wn = 0.5
betawisres['Num'] = (wp + wn) * betawisres['Phen Pos Fidelity'] * (1 - betawisres['Phen Neg Fidelity'])
betawisres['Denom'] = wp * (1 - betawisres['Phen Neg Fidelity']) + wn * betawisres['Phen Pos Fidelity']
betawisres['Phenomenon Characterization Score'] = betawisres['Num'] / betawisres['Denom']
betawisres['Num'] = (wp + wn) * betawisres['Model Pos Fidelity'] * (1 - betawisres['Model Neg Fidelity'])
betawisres['Denom'] = wp * (1 - betawisres['Model Neg Fidelity']) + wn * betawisres['Model Pos Fidelity']
betawisres['Model Characterization Score'] = betawisres['Num'] / betawisres['Denom']
betawisres.head()

,Dataset,Explainer,Run,Phen Pos Fidelity,Phen Neg Fidelity,Model Pos Fidelity,Model Neg Fidelity,Unfaithfulness,Exp Size Percent,Num,Denom,Phenomenon Characterization Score,Model Characterization Score
0,Texas,GNN,11,0.032787,0.0,0.038251,0.0,1.0,0.44,0.038251,0.519126,0.063492,0.073684


In [5]:
betawisres = pd.read_csv('data/Heterophilic/WisconsinBetaRunConsistancy.csv')
betawisres = betawisres[betawisres['Run'] == 22]
wp = 0.5
wn = 0.5
betawisres['Num'] = (wp + wn) * betawisres['Phen Pos Fidelity'] * (1 - betawisres['Phen Neg Fidelity'])
betawisres['Denom'] = wp * (1 - betawisres['Phen Neg Fidelity']) + wn * betawisres['Phen Pos Fidelity']
betawisres['Phenomenon Characterization Score'] = betawisres['Num'] / betawisres['Denom']
betawisres['Num'] = (wp + wn) * betawisres['Model Pos Fidelity'] * (1 - betawisres['Model Neg Fidelity'])
betawisres['Denom'] = wp * (1 - betawisres['Model Neg Fidelity']) + wn * betawisres['Model Pos Fidelity']
betawisres['Model Characterization Score'] = betawisres['Num'] / betawisres['Denom']
betawisres.head()

,Dataset,Explainer,Run,Phen Pos Fidelity,Phen Neg Fidelity,Model Pos Fidelity,Model Neg Fidelity,Unfaithfulness,Exp Size Percent,Num,Denom,Phenomenon Characterization Score,Model Characterization Score
0,Wisconsin,Beta,22,0.027888,0.003984,0.031873,0.003984,0.999952,0.419417,0.031746,0.513944,0.054258,0.061768


In [6]:
betawisres = pd.read_csv('data/Heterophilic/WisconsinGNNRunConsistancy.csv')
betawisres = betawisres[betawisres['Run'] == 39]
wp = 0.5
wn = 0.5
betawisres['Num'] = (wp + wn) * betawisres['Phen Pos Fidelity'] * (1 - betawisres['Phen Neg Fidelity'])
betawisres['Denom'] = wp * (1 - betawisres['Phen Neg Fidelity']) + wn * betawisres['Phen Pos Fidelity']
betawisres['Phenomenon Characterization Score'] = betawisres['Num'] / betawisres['Denom']
betawisres['Num'] = (wp + wn) * betawisres['Model Pos Fidelity'] * (1 - betawisres['Model Neg Fidelity'])
betawisres['Denom'] = wp * (1 - betawisres['Model Neg Fidelity']) + wn * betawisres['Model Pos Fidelity']
betawisres['Model Characterization Score'] = betawisres['Num'] / betawisres['Denom']
betawisres.head()

,Dataset,Explainer,Run,Phen Pos Fidelity,Phen Neg Fidelity,Model Pos Fidelity,Model Neg Fidelity,Unfaithfulness,Exp Size Percent,Num,Denom,Phenomenon Characterization Score,Model Characterization Score
0,Wisconsin,GNN,39,0.031873,0.007968,0.031873,0.007968,0.999667,0.487379,0.031619,0.511952,0.061761,0.061761


In [7]:
#ec1 = '#1A85FF'
ec1 = '#000000'

In [8]:
np.unique(wisout)

array([0, 1, 2, 3, 4])

In [9]:
np.unique(texout)

array([0, 1, 2, 3, 4])

In [10]:
wisout = np.load('data/Other/WisconsinNodeLabels.npy')
texout = np.load('data/Other/TexasNodeLabels.npy')

In [11]:
color = dict()
color[0] = '#7b3294'
color[1] = '#c2a5cf'
color[2] = '#000000'
color[3] = '#a6dba0'
color[4] = '#008837'

In [12]:
g = Network(notebook=True, directed=True)
ei1 = list(gnnwisgraph['P1'])
ei2 = list(gnnwisgraph['P2'])
prob = list(gnnwisgraph['Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append(color[wisout[i]])
for i in u2:
    if i not in n:
        n.append(i)
        c.append(color[wisout[i]])
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    edge['color'] = ec1
g.show('data/Vis/WisconsinNodesGraphGNN.html')

data/Vis/WisconsinNodesGraphGNN.html


In [13]:
g = Network(notebook=True, directed=True)
ei1 = list(betawisgraph['P1'])
ei2 = list(betawisgraph['P2'])
prob = list(betawisgraph['Probability'])
b1 = []
b2 = []
n = []
c = []
width = dict()
wtr = set()
mx = 0
mn = 1
for i in range(len(ei1)):
    if prob[i] >= 0.55:
        if prob[i] > mx:
            mx = prob[i]
        if prob[i] < mn:
            mn = prob[i]
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append(color[wisout[i]])
for i in u2:
    if i not in n:
        n.append(i)
        c.append(color[wisout[i]])
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    calc = (width[b1[i]][b2[i]] - mn + 1e-5) / (mx - mn)
    w = 5 * calc
    if w <= 0:
        print(w)
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    edge['color'] = 'black'
g.show('data/Vis/WisconsinNodesGraphBetaProb.html')

data/Vis/WisconsinNodesGraphBetaProb.html


In [14]:
g = Network(notebook=True, directed=True)
ei1 = list(gnntexgraph['P1'])
ei2 = list(gnntexgraph['P2'])
prob = list(gnntexgraph['Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append(color[texout[i]])
for i in u2:
    if i not in n:
        n.append(i)
        c.append(color[texout[i]])
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    edge["color"] = ec1
g.show('data/Vis/TexasNodesGraphGNN.html')

data/Vis/TexasNodesGraphGNN.html


In [15]:
g = Network(notebook=True, directed=True)
ei1 = list(betatexgraph['P1'])
ei2 = list(betatexgraph['P2'])
prob = list(betatexgraph['Probability'])
b1 = []
b2 = []
n = []
c = []
width = dict()
wtr = set()
lst = []
mx = 0
mn = 1
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        if prob[i] > mx:
            mx = prob[i]
        if prob[i] < mn:
            mn = prob[i]
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            lst.append(prob[i])
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append(color[texout[i]])
for i in u2:
    if i not in n:
        n.append(i)
        c.append(color[texout[i]])
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    calc = (width[b1[i]][b2[i]] - mn + 1e-5) / (mx - mn)
    w = 5 * calc
    if w <= 0:
        print(w)
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    edge['color'] = 'black'
g.show('data/Vis/TexasNodesGraphBetaProb.html')

data/Vis/TexasNodesGraphBetaProb.html
